In [ ]:
#!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U bitsandbytes
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0
!pip3 install -q -U rouge_score

import os
os._exit(0)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━

In [1]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [2]:
import gc
import tqdm
import copy
import time


import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd


In [ ]:
# number1 = 10
# def _add(a, b):
#   return a+b
#
# @ray.remote
# def test_func_with_print(number2):
#   global number1
#   res = _add(number2, number1)
#   print(f"The res is: {res}")
#   for i in tqdm.tqdm(range(5)):
#     time.sleep(1)
#     continue
#   return res
#
# id = test_func_with_print.remote(10)
# print("Ray ID:",id)
# ray.get(id)

In [5]:
### Load dataset
train_set = load_dataset("rPucs/TripletDollyQA-3k-Gemma-Nlg", "train")["train"]
test_set = load_dataset("rPucs/TripletDollyQA-3k-Gemma-Nlg", "test")["train"]
test_set = test_set.select(range(0,100))

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

63


In [ ]:
### Input paramteres
model_ids = [
    #("google/gemma-2b", "gemma"),
    #("google/gemma-2b-it", "gemma"),
    #("google/gemma-7b", "gemma"),
    #("google/gemma-7b-it", "gemma"),
    #("meta-llama/Llama-2-7b", "llama"),
    #("meta-llama/Llama-2-7b-hf", "llama"),
    #("meta-llama/Llama-2-7b-chat", "llama"),
    #("meta-llama/Llama-2-7b-chat-hf", "llama"),
    #("mistralai/Mistral-7B-v0.1", "mistral"),
    #("mistralai/Mistral-7B-Instruct-v0.1", "mistral"),
    #("mistralai/Mistral-7B-Instruct-v0.2", "mistral"),
    #("rPucs/Mistral-7B-Instruct-v0.2TripletDolly-Rebel", "mistral"),
    #("rPucs/gemma-2b-itTripletDolly-Rebel", "gemma"),
    #("rPucs/gemma-7b-itTripletDolly-Rebel", "gemma"),
    ("rPucs/gemma-2b-itTripletDolly-WebNLG", "gemmanlg")
]
templates = {
    "gemma": """<start_of_turn>user
{query} <end_of_turn>\n""",
    "llama":"",
    "mistral":"""
<s>
[INST]
{query}
[/INST]
</s>

"""
}



In [ ]:
def get_completion(prompt: str, model, tokenizer) -> str:

  device = "cuda:0"
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

# only completion
def get_completion_batch(prompt: list[str], model, tokenizer, cut_prompt=True) -> list[str]:
  device = "cuda:0"
  encodeds = tokenizer(prompt,\
                      return_tensors="pt", add_special_tokens=True,\
                      padding=True)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=600, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)

  if cut_prompt
    cut_decoded = []
    for i, response in enumerate(decoded):
      cut_decoded.append( (response.split(prompt[i])[1] if prompt[i] in response else response) )
    decoded = cut_decoded
  return decoded

def context_to_string(contexts):
  contexts = [f"The relation {context['type']} of {context['head']} is {context['tail']}." for context in contexts]
  return " ".join(contexts)

def get_full_input_query(input, contexts):
  return f"""
{input}
Extra data:
{context_to_string(contexts)}
"""

def generate_prompt(query: str, simpleModelName : str, appendOutput : bool, out: str | None = None):
  prefix_text = 'Below is an instruction or question that describes a task or requires an answer.'\
  'Write a response that appropriately completes the request. \n\n'

  # Samples with additional context into.
  if simpleModelName == "gemma":
    text = templates["gemma"]
    end = "\n<start_of_turn>model{query}\n<end_of_turn>"
  elif simpleModelName == "mistral":
    text = templates["mistral"]
    end = "<s>\n{query}\n</s>"
  else:
    text = templates["mistral"]
    end = "<s>\n{query}\n</s>"

  query = prefix_text + query
  text = text.format(query=query)

  if appendOutput:
    assert out != None
    text = text + end.format(query=out)

  return text

def generate_prompt_batch(queries: list[str], simpleModelName : str, appendOutput : bool, expected_responses: list[str] | None = None):
  if expected_responses:
    assert len(queries) == len(expected_responses, "length of input and output lists do not match")
    return [generate_prompt(q, simpleModelName, appendOutput, o) for (q,o) in zip(queries, expected_responses)]
  return [generate_prompt(q, simpleModelName, appendOutput=False) for q in queries]

def get_max_new_tokens(dataset, tokenizer):
  max_new_token = 0
  for response in dataset["response"]:
    encodeds = tokenizer(response, return_tensors="pt", add_special_tokens=True)
    max_new_token = max(max_new_token, encodeds["input_ids"].shape[-1])
  return max_new_token

SyntaxError: expected ':' (<ipython-input-47-756b26bb760e>, line 21)

In [ ]:
#for i in range(dataset.num_rows):
#  inp = get_full_input_query(dataset[i]["instruction"], dataset[i]["context"])
#  prompt = generate_prompt(query=inp, simpleModelName="mistral", appendOutput=False)
#  print(prompt)
#  break
#tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b",
#                                          #add_eos_token=True
#                                          #padding_side="left"
#                                          )
#max_new_tokens = get_max_new_tokens(test_set, tokenizer)
#print(max_new_tokens)

# 2. Evaluate Raw

In [ ]:
def evaluate(model_id, simple_name, evaluation_name_id, batch_size=1):
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quent_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )
  model = AutoModelForCausalLM.from_pretrained(model_id,
                                               quantization_config=bnb_config,
                                               device_map="auto")
  tokenizer = AutoTokenizer.from_pretrained(model_id,
                                            add_eos_token=True,
                                            padding_side="left"
                                            )
  if simple_name in ["llama", "mistral"]:
    tokenizer.pad_token = tokenizer.eos_token

  # test_set
  eval_data = copy.deepcopy(test_set)
  eval_data = eval_data.add_column("prompt", [generate_prompt( get_full_input_query( datapoint["instruction"], datapoint["context"] ), simpleModelName=simple_name, appendOutput=False ) for datapoint in eval_data] )
  model_responses = []
  for i in tqdm.tqdm(range(0, eval_data.num_rows, batch_size)):
    completions = get_completion_batch( eval_data[i:i+batch_size]["prompt"], model, tokenizer )
    model_responses = model_responses + completions
  eval_data = eval_data.add_column("model_reponse",  model_responses)

  # release gpu ram
  del model, tokenizer
  torch.cuda.empty_cache()

  rouge = load_metric('rouge')
  references = list(eval_data['response'])
  predictions = list(eval_data['model_reponse'])
  results = rouge.compute(predictions=predictions, references=references)

  df = rouge_to_pd(results, evaluation_name_id)
  df.to_csv(evaluation_name_id + ".csv", index=False)

  del eval_data, df
  torch.cuda.empty_cache()


def rouge_to_pd(r, IDName: str):
    """
    Converts ROUGE scores into a pandas DataFrame.

    Parameters:
    - r: A dictionary containing ROUGE scores. Each key is a ROUGE method and each value is a tuple of tuples
         containing precision, recall, and f-measure for low, mid, and high estimations.
    - IDName: A string that identifies the set of ROUGE scores.

    Returns:
    - A pandas DataFrame with columns for the IDName, method, and each metric (low, mid, high precision, recall, f-measure).
    """
    data = []
    # Fixed: Split the string into a list of strings
    for method in ["rouge1", "rouge2", "rougeL", "rougeLsum"]:
        ((low_precision, low_recall, low_fmeasure),
         (mid_precision, mid_recall, mid_fmeasure),
         (high_precision, high_recall, high_fmeasure)) = r[method]
        data.append({
            "IDName": IDName,
            "method": method,
            "low_precision": low_precision,
            "low_recall": low_recall,
            "low_fmeasure": low_fmeasure,
            "mid_precision": mid_precision,
            "mid_recall": mid_recall,
            "mid_fmeasure": mid_fmeasure,
            "high_precision": high_precision,
            "high_recall": high_recall,
            "high_fmeasure": high_fmeasure
        })
    return pd.DataFrame(data)

# rouge_to_pd(d, "Teszt")

In [ ]:
%%time

batch_size = 4
print(f"Length of eval data: {len(test_set)}")

for model_id, simple_name in model_ids:
  print("Next model: {}".format(model_id))
  try:
    evaluate(model_id=model_id, simple_name=simple_name, evaluation_name_id= ("finetuned_rebel_"+model_id).replace("/", "_"), batch_size=batch_size )
    #rayRef = evaluate.remote(model_id=model_id, simple_name=simple_name, evaluation_name_id= ("raweval_"+model_id).replace("/", "_"), batch_size=batch_size )
    #ray.get(rayRef)
  except Exception as e:
    print(f"An error occurred when running:{model_id} {simple_name}: {e}")


# Test inference

In [ ]:
def test(model_id, simple_name, evaluation_name_id, test_data, batch_size=1):
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quent_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )
  model = AutoModelForCausalLM.from_pretrained(model_id,
                                               quantization_config=bnb_config,
                                               device_map="auto")
  tokenizer = AutoTokenizer.from_pretrained(model_id,
                                            add_eos_token=True,
                                            padding_side="left"
                                            )
  if simple_name in ["gemma", "llama", "mistral"]:
    tokenizer.pad_token = tokenizer.eos_token

  # test_set
  eval_data = copy.deepcopy(test_data)
  eval_data = eval_data.add_column("prompt", [generate_prompt( get_full_input_query( datapoint["instruction"], datapoint["context"] ), simpleModelName=simple_name, appendOutput=False ) for datapoint in eval_data] )
  model_responses = []
  for i in tqdm.tqdm(range(0, eval_data.num_rows, batch_size)):
    completions = get_completion_batch( eval_data[i:i+batch_size]["prompt"], model, tokenizer )
    model_responses = model_responses + completions
  eval_data = eval_data.add_column("model_response",  model_responses)

  # release gpu ram
  del model, tokenizer
  torch.cuda.empty_cache()

  return eval_data["prompt"], eval_data["model_response"]


In [ ]:
sample_dataset = test_set.select([0,1,2,3])


batch_size = 2
print(f"Length of eval data: {len(test_set)}")

results = {}
for i, (model_id, simple_name) in enumerate(model_ids):
  print("Next model: {}".format(model_id))
  print(f"Progress: {i}/{len(model_ids)}")
  prompts, responses = test(model_id=model_id, simple_name=simple_name, evaluation_name_id= ("finetuned_rebel_"+model_id).replace("/", "_"), test_data=sample_dataset, batch_size=batch_size )
  results[model_id] = (prompts, responses)

Length of eval data: 100
Next model: google/gemma-2b
Progress: 0/7


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [01:26<00:00, 43.21s/it]


Next model: google/gemma-2b-it
Progress: 1/7


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [01:25<00:00, 42.95s/it]


Next model: rPucs/gemma-7b-itTripletDolly-Rebel
Progress: 6/7


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 2/2 [02:07<00:00, 63.95s/it]


In [ ]:
df = []
for name, (prompts, responses) in results.items():
  df = df + [(name, prompt, response[len(prompt):]) for (prompt,response) in zip(prompts,responses)]
#df = {"model":df[0], "prompt":df[1], "response":df[2]}
df = pd.DataFrame(df, columns=['model', 'prompt', 'response'])

In [ ]:
df

,model,prompt,response
0,google/gemma-2b,<start_of_turn>user\nBelow is an instruction o...,urn>\n<eos>\n2016 | All Rights Reserved 2020<...
1,google/gemma-2b,<start_of_turn>user\nBelow is an instruction o...,located in the administrative territorial enti...
2,google/gemma-2b,<start_of_turn>user\nBelow is an instruction o...,urn>\n<eos>.R.C.000029685201702602999332942888...
3,google/gemma-2b,<start_of_turn>user\nBelow is an instruction o...,><pad><pad><pad><pad><pad><pad><pad><pad><pad>...
4,google/gemma-2b-it,<start_of_turn>user\nBelow is an instruction o...,urn>\n<eos>{First Era} ({1915-1945}) \n{Third ...
5,google/gemma-2b-it,<start_of_turn>user\nBelow is an instruction o...,located in the administrative territorial enti...
6,google/gemma-2b-it,<start_of_turn>user\nBelow is an instruction o...,urn>\n<eos> An electric field is produced when...
7,google/gemma-2b-it,<start_of_turn>user\nBelow is an instruction o...,><pad><pad><pad><pad><pad><pad><pad><pad><pad>...
8,google/gemma-7b-it,<start_of_turn>user\nBelow is an instruction o...,urn>\n<eos>**Response:**\n\nThe text does not ...
9,google/gemma-7b-it,<start_of_turn>user\nBelow is an instruction o...,located in the administrative territorial enti...


In [ ]:
df.to_excel("./data.xlsx")

In [ ]:
for i, (model_id, simple_name) in enumerate(model_ids):
  print(model_id)

google/gemma-2b
google/gemma-2b-it
google/gemma-7b-it
mistralai/Mistral-7B-Instruct-v0.2
rPucs/Mistral-7B-Instruct-v0.2TripletDolly-Rebel
rPucs/gemma-2b-itTripletDolly-Rebel
rPucs/gemma-7b-itTripletDolly-Rebel


notes:
gemma it and mistral it v2 faster inference